# Dataset: Base de microdatos EVyTH
  
- [Sitio Web](https://datos.gob.ar/dataset/turismo-encuesta-viajes-turismo-hogares-evyth---microdatos/archivo/turismo_645e5505-68ee-4cfa-90f9-fcc9a4a34a85)

- [Descarga CSV](http://datos.yvera.gob.ar/dataset/b5819e9b-5edf-4aad-bd39-a81158a2b3f3/resource/645e5505-68ee-4cfa-90f9-fcc9a4a34a85/download/evyth_microdatos.csv)

- [PDF informativo](http://datos.yvera.gob.ar/dataset/b5819e9b-5edf-4aad-bd39-a81158a2b3f3/resource/1b5d26f1-0ab9-4908-a44f-9ac5bd685f1b/download/evyth-data.pdf)

- [Información en TXT](https://datos.yvera.gob.ar/dataset/b5819e9b-5edf-4aad-bd39-a81158a2b3f3/resource/20e2c018-a2ee-4d97-9c67-a4303f669255/download/evyth_diccionario_registro.txt)

- [Diccionario en CSV](http://datos.yvera.gob.ar/dataset/b5819e9b-5edf-4aad-bd39-a81158a2b3f3/resource/d8107236-f5ba-428d-8a31-f9b2034d8c8f/download/evyth_diccionario_registro.csv)
        

### Librerías

In [1]:
import pandas as pd
import numpy as np

### Carga de datos

In [2]:
data = pd.read_csv('data/evyth_microdatos.csv')
dicc = pd.read_csv('data/evyth_diccionario_registro.csv')

### Inspección de dataset

In [3]:
#data.info()

In [4]:
#data.head()

In [5]:
#dicc.info()

In [6]:
#dicc.tail()

## Tratamiento de datos

### Diccionario de registros

Se realiza una limpieza de columnas y filas, para dejar solamente la información útil para el caso de estudio

In [7]:
# De inicio se puede ver que el dataset posee valores NaN. Esto significa un 0 u opción 0.
dicc.isna().sum()

variable        0
opcion         89
descripcion     0
dtype: int64

In [8]:
# Se procede entonces a reemplazar estos valores en la columna de "opcion"
dicc.opcion.fillna(0,inplace=True)
dicc.isna().sum()

variable       0
opcion         0
descripcion    0
dtype: int64

In [9]:
# Se cambia el formato de la columna de opcion a enteros, para mayor legibilidad.
# Esto permite comparar directamente los valores con la otra tabla
dicc['opcion'] = dicc['opcion'].astype('int16')
dicc.dtypes

variable       object
opcion          int16
descripcion    object
dtype: object

### Dataset principal

- ##### Reducción de número de columnas
El resultado final, implica reducir el número de columnas, que van a ser luego las variables ingresadas en el formulario.


- ##### Reducción de número de filas
La idea es generar un dataset de hasta 50 participantes, para no llenar una gran cantidad de formularios de Google. El objetivo no es el análisis completo en sí, si no prestar atención a todo el proceso.

#### Reducción de número de columnas

Se genera un nuevo dataset, solo con las columnas de interés, a partir del diccionario de registro

In [10]:
data = data[['anio','region_origen','aglomerado_origen','localidad_destino','tipo_visitante', 'px08','px09','px10_1','px17_2_5','px17_2_11']]

Como en el caso del dataset diccionario, se revisa por valores nulos

In [11]:
data.isna().sum()

anio                   0
region_origen          0
aglomerado_origen      0
localidad_destino      0
tipo_visitante         0
px08                   0
px09                   0
px10_1                 0
px17_2_5             406
px17_2_11            410
dtype: int64

In [12]:
data['px17_2_5'].fillna(0,inplace=True)
data['px17_2_11'].fillna(0,inplace=True)

Las columnas con valores float, también se pasan a enteros

In [13]:
data.dtypes

anio                   int64
region_origen          int64
aglomerado_origen      int64
localidad_destino     object
tipo_visitante         int64
px08                   int64
px09                   int64
px10_1                 int64
px17_2_5             float64
px17_2_11            float64
dtype: object

In [14]:
data['px17_2_5'] = data['px17_2_5'].astype('int16')
data['px17_2_11'] = data['px17_2_11'].astype('int16')

In [15]:
data.dtypes

anio                  int64
region_origen         int64
aglomerado_origen     int64
localidad_destino    object
tipo_visitante        int64
px08                  int64
px09                  int64
px10_1                int64
px17_2_5              int16
px17_2_11             int16
dtype: object

#### Reducción de número de filas
Para esto se seleccionan solamente los participantes:

    - solo del 2022
    - que hayan viajado en avion
    - con origen: que no sean de la provincia de Buenos Aires 
    - como destino:  Bariloche

Para ello se quieren obtener los códigos correspondientes, de la tabla de diccionario, y así poder mapearlos en la tabla de datos

In [16]:
dicc.head()

,variable,opcion,descripcion
0,id_hogar,0,Código de identificación único de hogares
1,id_viajes,0,Código de identificación único de viajes
2,miembro,0,Número de miembro en el hogar
3,anio,0,Año
4,trimestre,0,Trimestre


In [17]:
# Los que hayan viajado en avión
dicc[dicc['descripcion'] == "Avión"]

,variable,opcion,descripcion
134,px09,4,Avión


In [18]:
# Provenientes de todo el país, excepto de provincia de Buenos Aires (solo para reducir el dataset final)
# Se obtienen los números de opciones a excluir
regiones = dicc[(dicc['descripcion'].str.contains('Buenos Aires')) & (dicc['variable'] == 'region_origen')]

In [19]:

data = data[(data['anio'] == 2022) & (data['localidad_destino'] == 'San Carlos de Bariloche') & (data['px09'] == 4) & (data['region_origen'] > 3)]
#Dado que ahora todas las entradas son del año 2022, que hayan viajdo en avión a Bariloche, dicha columnas ya no son necesarias
data = data.drop(['anio','localidad_destino','px09'], axis=1)

In [20]:
# El resultado es un dataset de 41 filas y 12 columnas (cumpliendo con la expectativa de menos de 50 participantes)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41 entries, 396895 to 419939
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   region_origen      41 non-null     int64
 1   aglomerado_origen  41 non-null     int64
 2   tipo_visitante     41 non-null     int64
 3   px08               41 non-null     int64
 4   px10_1             41 non-null     int64
 5   px17_2_5           41 non-null     int16
 6   px17_2_11          41 non-null     int16
dtypes: int16(2), int64(5)
memory usage: 2.1 KB


#### Conversión de valores

Debido al gran número de entradas de la tabla, la misma hace uso de un archivo de texto con los metadatos, para obtener información de los códigos usados en cada variable. La mayoría de los datos son entonces de tipo int64. Tanto estos como los float64, pasarán a ser strings ahora, indicando el contenido real correspondiente.


Como se puede observar, la tabla "dicc" está ordenada (o tidy), respetando los tres principios básicos:

- cada variables es una columna
- cada fila es una observación
- cada celda es un valor individual de la tabla

En cambio, en la tabla "data" no se ve lo mismo, viéndose los valores de las variables en diversas columnas.
Para poder hacer trabajar mejor con dicho dataframe, se realiza una transformación.

In [21]:
dicc.head()

,variable,opcion,descripcion
0,id_hogar,0,Código de identificación único de hogares
1,id_viajes,0,Código de identificación único de viajes
2,miembro,0,Número de miembro en el hogar
3,anio,0,Año
4,trimestre,0,Trimestre


In [22]:
data.head()

,region_origen,aglomerado_origen,tipo_visitante,px08,px10_1,px17_2_5,px17_2_11
396895,4,13,1,7,2,2,2
397269,5,4,1,7,1,2,1
398377,7,10,1,3,2,1,2
399841,7,10,1,7,1,2,1
400917,4,13,1,8,2,1,1


In [23]:
# Se reinician los índices
data = data.reset_index(drop=True)

Se reemplazan ahora por los valores respectivos

In [24]:
for col in data.columns:
    if col != 'localidad_destino':
        for row in data[col]:
            desc = dicc[(dicc['variable'] == col) & (dicc['opcion'] == int(row))]['descripcion'].values[0]
            data[col] = data[col].replace(row, desc)

In [25]:
# Primero, se renombran las columnas, para ser más entendibles
data = data.rename(columns={'px08':'alojamiento','px10_1':'motivo_viaje','px17_2_5':'aire_libre','px17_2_11':'naturaleza'})

,region_origen,aglomerado_origen,tipo_visitante,alojamiento,motivo_viaje,aire_libre,naturaleza
0,Centro,Gran Córdoba,Turista,Hotel u hostería hasta 3 estrellas (o categori...,Visitas a familiares o amigos,No,No
1,Litoral,Gran Rosario,Turista,Hotel u hostería hasta 3 estrellas (o categori...,"Esparcimiento, ocio, recreacion",No,Si
2,Cuyo,Gran Mendoza,Turista,Vivienda de familiares o amigos que residen e...,Visitas a familiares o amigos,Si,No
3,Cuyo,Gran Mendoza,Turista,Hotel u hostería hasta 3 estrellas (o categori...,"Esparcimiento, ocio, recreacion",No,Si
4,Centro,Gran Córdoba,Turista,"Hotel 4 y 5 estrellas, apart hotel o similares",Visitas a familiares o amigos,Si,Si
5,Centro,Gran Córdoba,Turista,Hotel u hostería hasta 3 estrellas (o categori...,"Esparcimiento, ocio, recreacion",No,Si
6,Centro,Gran Córdoba,Turista,Hotel u hostería hasta 3 estrellas (o categori...,"Esparcimiento, ocio, recreacion",No,Si
7,Litoral,Gran Rosario,Turista,Hotel u hostería hasta 3 estrellas (o categori...,"Esparcimiento, ocio, recreacion",No,Si
8,Litoral,Gran Rosario,Turista,Hotel u hostería hasta 3 estrellas (o categori...,"Esparcimiento, ocio, recreacion",No,Si
9,Litoral,Concordia,Turista,"Camping (carpa, casa rodante)","Esparcimiento, ocio, recreacion",Si,Si


In [26]:
# Para resumir aún más la tabla, dado que todos los visitantes son considerados como turistas, se quita la columna

data = data.drop('tipo_visitante', axis=1)
data


,region_origen,aglomerado_origen,alojamiento,motivo_viaje,aire_libre,naturaleza
0,Centro,Gran Córdoba,Hotel u hostería hasta 3 estrellas (o categori...,Visitas a familiares o amigos,No,No
1,Litoral,Gran Rosario,Hotel u hostería hasta 3 estrellas (o categori...,"Esparcimiento, ocio, recreacion",No,Si
2,Cuyo,Gran Mendoza,Vivienda de familiares o amigos que residen e...,Visitas a familiares o amigos,Si,No
3,Cuyo,Gran Mendoza,Hotel u hostería hasta 3 estrellas (o categori...,"Esparcimiento, ocio, recreacion",No,Si
4,Centro,Gran Córdoba,"Hotel 4 y 5 estrellas, apart hotel o similares",Visitas a familiares o amigos,Si,Si
5,Centro,Gran Córdoba,Hotel u hostería hasta 3 estrellas (o categori...,"Esparcimiento, ocio, recreacion",No,Si
6,Centro,Gran Córdoba,Hotel u hostería hasta 3 estrellas (o categori...,"Esparcimiento, ocio, recreacion",No,Si
7,Litoral,Gran Rosario,Hotel u hostería hasta 3 estrellas (o categori...,"Esparcimiento, ocio, recreacion",No,Si
8,Litoral,Gran Rosario,Hotel u hostería hasta 3 estrellas (o categori...,"Esparcimiento, ocio, recreacion",No,Si
9,Litoral,Concordia,"Camping (carpa, casa rodante)","Esparcimiento, ocio, recreacion",Si,Si


Se guarda el resultado final en un nuevo CSV

In [27]:
data.to_csv('data/output_viajes.csv')